<a href="https://colab.research.google.com/github/aswinaus/Reinforcement-Learning/blob/main/Agentic_RewardFunction_GRPO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install llama-index -q
!pip install langchain -q
!pip install langchain_experimental -q
!pip install jedi
!pip install --upgrade openai -q

In [ ]:
from openai import OpenAI
client = OpenAI()

response = client.responses.create(
    model="gpt-5",
    input="What is Form990 EZ and when should an organiaztion complete Form990 EZ form? And how is it different from Schedule H? Can you show the results in side by side comparison table with headers and also link to the document"
)

print(response.output_text)


In [ ]:
import os
import nest_asyncio
nest_asyncio.apply()

In [ ]:
from google.colab import userdata
# Set the OpenAI API key as an environment variable
os.environ["OPENAI_API_KEY"] =  userdata.get('OPENAI_API_KEY')

In [ ]:
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings
# Setup OpenAI Model and Embeddings used for indexing the documents
Settings.llm = OpenAI(model='gpt-4o-mini', temperature=0.2)
Settings.embed_model = OpenAIEmbedding(model='text-embedding-3-small')
Settings.chunk_size = 1024

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
data_dir = '/content/drive/MyDrive' # Input a data dir path from your mounted Google Drive


import torch
import torch.nn as nn # Explicitly import torch.nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
from torch.distributions import Normal
import os # Import os to check for existing index
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, SummaryIndex, StorageContext, load_index_from_storage, Settings # Import necessary LlamaIndex components
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from google.colab import userdata

In [ ]:
# Load OECD guidelines documents for Transfer Pricing
docs_OECD_guidelines = SimpleDirectoryReader(f"{data_dir}/RAG/data/OECD/").load_data()
# Load OECD guidelines documents for Form990
docs_Form990_guidelines = SimpleDirectoryReader(f"{data_dir}/RAG/data/Form990/").load_data()

In [ ]:
# In order to avoid repeated calls to LLMs we can store the documents index and load it if present else create it
PERSIST_INDEX_DIR = f"/{data_dir}/RAG/data/"
def get_index(index_name, doc_file_path):
  index = None
  if not os.path.exists(f"{PERSIST_INDEX_DIR}{index_name}/"):
    # Load the documents
    documents = SimpleDirectoryReader(input_files=[doc_file_path]).load_data()
    index = VectorStoreIndex.from_documents(documents)
    # Store the index to disk
    index.storage_context.persist(f"{PERSIST_INDEX_DIR}{index_name}/")
  else: # Load index from disk
    storage_context = StorageContext.from_defaults(persist_dir=f"{PERSIST_INDEX_DIR}{index_name}/")
    index = load_index_from_storage(storage_context)

  return index

In [ ]:
#initialise a storage context and use that for both Vector Index and Summary Index for OECD
#split the OECD document into multiple nodes
oecd_nodes = Settings.node_parser.get_nodes_from_documents(docs_OECD_guidelines)
#split the Form990 document into multiple nodes
form990_nodes = Settings.node_parser.get_nodes_from_documents(docs_Form990_guidelines)

storage_context = StorageContext.from_defaults()

storage_context.docstore.add_documents(oecd_nodes)
storage_context.docstore.add_documents(form990_nodes)
# Setup Vector and Summary Index from Storage Context
summary_index = SummaryIndex(oecd_nodes, storage_context=storage_context)
vector_index = VectorStoreIndex(oecd_nodes, storage_context=storage_context)

# Setup Indices.In order to avoid repeated calls to LLMs we can store the documents index and load it if present else create it
OECD_index = get_index("OECDTPGuidelines",f"{data_dir}/RAG/data/OECD/OECD_Transfer_Pricing_Guidelines.pdf")
form990_guidelines_index = get_index("Form990Guidelines",f"{data_dir}/RAG/data/Form990/Form990_Guidelines.pdf")


In [ ]:
!pip install llama-index -q
!pip install langchain -q
!pip install langchain_experimental -q

In [ ]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata
import json # Import the json module for schema definition

# Redefine or ensure tools are available from previous successful cells
# Assuming OECD_query_tool and Form990_query_tool are defined here or in a prior cell
# Example definition if not already available:
# OECD_engine = OECD_index.as_query_engine(similarity_top_k=3) # Assuming OECD_index is loaded
# form990_guidelines_engine = form990_guidelines_index.as_query_engine(similarity_top_k=3) # Assuming form990_guidelines_index is loaded
# OECD_query_tool = QueryEngineTool(
#                       query_engine=OECD_engine,
#                       metadata=ToolMetadata(
#                           name="OECD_QueryEngineTool_2022",
#                           description="Provides information about Transfer Pricing Guidelines for Organization from OECD for year 2022"
#                       )
#                     )
# Form990_query_tool = QueryEngineTool(
#                       query_engine=form990_guidelines_engine,
#                       metadata=ToolMetadata(
#                           name="form990_2022",
#                           description="Provides information about Form990 filling guidelines for Non-Profit Organization only from the index which was set for Form990_Guidelines.pdf "
#                       )
#                     )
# tools = [OECD_query_tool, Form990_query_tool] # Ensure 'tools' list is defined

# 1. Define a function openai_tool_definition
def openai_tool_definition(query_engine_tool: QueryEngineTool) -> dict:
    """
    Converts a LlamaIndex QueryEngineTool to the OpenAI Chat Completions API
    tool format.

    Args:
        query_engine_tool: The QueryEngineTool instance.

    Returns:
        A dictionary representing the tool definition in OpenAI's format.
    """
    # 4. Define the parameters schema
    parameters_schema = {
        "type": "object",
        "properties": {
            "input": {
                "type": "string",
                "description": "The query string to pass to the tool."
            }
        },
        "required": ["input"],
    }

    # 2. Construct the dictionary for the tool definition
    tool_definition = {
        "type": "function",
        "function": {
            # 3. Extract name and description from metadata
            "name": query_engine_tool.metadata.name,
            "description": query_engine_tool.metadata.description,
            "parameters": parameters_schema,
        },
    }
    return tool_definition

# Create query engines
OECD_engine = OECD_index.as_query_engine(similarity_top_k=3) # Assuming OECD_index is loaded
form990_guidelines_engine = form990_guidelines_index.as_query_engine(similarity_top_k=3) # Assuming form990_guidelines_index is loaded

# Create QueryEngineTool instances
OECD_query_tool = QueryEngineTool(
                      query_engine=OECD_engine,
                      metadata=ToolMetadata(
                          name="OECD_QueryEngineTool_2022",
                          description="Provides information about Transfer Pricing Guidelines for Organization from OECD for year 2022"
                      )
                    )
Form990_query_tool = QueryEngineTool(
                      query_engine=form990_guidelines_engine,
                      metadata=ToolMetadata(
                          name="form990_2022",
                          description="Provides information about Form990 filling guidelines for Non-Profit Organization only from the index which was set for Form990_Guidelines.pdf "
                      )
                    )
# Create the tools list
tools = [OECD_query_tool, Form990_query_tool]


# 5. Apply this function to existing tools
# Ensure 'tools' list is available from previous cells
# If not, you'd need to define OECD_query_tool and Form990_query_tool here
# Assuming 'tools' is defined and contains OECD_query_tool and Form990_query_tool

if 'tools' in globals() and isinstance(tools, list) and len(tools) > 0:
    openai_tools = [openai_tool_definition(tool) for tool in tools]
    print("OpenAI tool definitions created successfully:")
    # Print the created tool definitions for verification
    print(json.dumps(openai_tools, indent=2))
else:
    print("Error: 'tools' list not found or is empty. Cannot create OpenAI tool definitions.")
    openai_tools = [] # Initialize as empty if tools are not available

## Implement chat completions logic

### Subtask:
Create a function or class that uses `openai.ChatCompletion.create` (or the equivalent using the `openai` library) to send messages to the model.


**Reasoning**:
Define a function `chat_with_tools` that uses `openai.ChatCompletion.create` (or the equivalent) to send messages and tools to the model and handle the response.



In [ ]:
import openai
import os # Import os to access environment variables

# Ensure the OpenAI API key is set
if "OPENAI_API_KEY" not in os.environ:
    print("Error: OPENAI_API_KEY environment variable not set.")
    # In a real application, you would handle this more robustly
    # For this example, we'll proceed but the API call will fail.


# 1. Define a Python function, for example chat_with_tools
def chat_with_tools(messages: list[dict], openai_tools: list[dict], model: str = "gpt-4o-mini", temperature: float = 0.7) -> dict:
    """
    Sends messages and OpenAI tool definitions to the Chat Completions API
    and returns the response.

    Args:
        messages: A list of message dictionaries in the OpenAI format.
        openai_tools: A list of tool definitions in the OpenAI format.
        model: The name of the OpenAI model to use (default: gpt-4o-mini).
        temperature: The sampling temperature (default: 0.7).

    Returns:
        A dictionary representing the response from the OpenAI API.
        Returns an empty dictionary and prints an error message if the API call fails.
    """
    try:
        # 2. Inside the function, use the openai.ChatCompletion.create method
        # Using the newer openai library syntax (v1.0+)
        # The older openai.ChatCompletion.create is deprecated in favor of client.chat.completions.create
        # Let's use the newer client syntax
        client = openai.OpenAI() # Assumes OPENAI_API_KEY is set as an environment variable

        response = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=openai_tools,
            tool_choice="auto",  # auto lets the model decide whether to call a tool or respond
            temperature=temperature,
        )
        # The response object structure is slightly different in the new client
        # Returning the full response object for now
        return response

    # 3. Include error handling (e.g., a try...except block) for the API call.
    except openai.APIError as e:
        print(f"OpenAI API error: {e}")
        return {}
    except Exception as e:
        print(f"An unexpected error occurred during the OpenAI API call: {e}")
        return {}

# Example usage (assuming 'openai_tools' is defined from the previous step)
# and 'messages' is a list of dictionaries like [{"role": "user", "content": "Your question here"}]
# Example messages setup:
# messages_history = [{"role": "user", "content": "What does Articles 9 of the OECD Model Tax Convention state?"}]
# if 'openai_tools' in globals() and openai_tools:
#     print("Attempting to call chat_with_tools...")
#     api_response = chat_with_tools(messages_history, openai_tools)
#     print("\nAPI Response:")
#     print(api_response)
# else:
#     print("openai_tools not defined. Skipping chat_with_tools example call.")


## Implement tool calling handling

### Subtask:
Within the chat loop, check the model's response for `tool_calls`. If present, execute the corresponding tool (query the appropriate LlamaIndex engine) and send the tool output back to the model.


**Reasoning**:
Implement the main chat loop that iteratively calls the chat_with_tools function, checks for tool_calls in the response, executes the tools using the corresponding LlamaIndex query engines, appends the tool outputs to the messages history, and continues the conversation until the model provides a final answer without tool calls. This combines steps 1-10 of the instructions.



In [ ]:
import json # Import json if not already imported in this block
import time # Import time for potential delays
import openai # Import openai

# Assume necessary variables and functions are defined from previous successful cells:
# OECD_index, form990_guidelines_index (if used), tools (LlamaIndex QueryEngineTool instances),
# openai_tools (OpenAI tool definitions), chat_with_tools, cosine_similarity_reward (if needed elsewhere)
# Settings (LlamaIndex) are assumed to be configured.

# Ensure query engines corresponding to the tools are accessible
# Assuming they are defined from prior steps, e.g.:
# OECD_engine = OECD_index.as_query_engine(similarity_top_k=3)
# form990_guidelines_engine = form990_guidelines_index.as_query_engine(similarity_top_k=3)

# Create a dictionary mapping tool names to query engines for easy lookup
# Ensure OECD_engine and form990_guidelines_engine are defined and loaded from index
query_engine_map = {}
if 'OECD_index' in globals() and OECD_index is not None:
    try:
        query_engine_map["OECD_QueryEngineTool_2022"] = OECD_index.as_query_engine(similarity_top_k=3)
        print("Created OECD query engine.")
    except Exception as e:
        print(f"Error creating OECD query engine: {e}")
else:
    print("OECD_index not found or loaded. OECD tool execution will fail.")

# Assuming form990_guidelines_index is also loaded and available
if 'form990_guidelines_index' in globals() and form990_guidelines_index is not None:
     try:
          query_engine_map["form990_2022"] = form990_guidelines_index.as_query_engine(similarity_top_k=3)
          print("Created Form990 query engine.")
     except Exception as e:
          print(f"Error creating Form990 query engine: {e}")
else:
     print("form990_guidelines_index not found or loaded. Form990 tool execution will fail.")


# 1. Initialize the conversation with a system message and the user's first question
messages = [{"role": "system", "content": "You are an assistant that provides answers to questions on OECD and Form990 using the available tools. Answer as accurately as possible based on the tool outputs. Whenever there is comparison make sure the results are in side by side comparison table with headers and add links to the document."}]
user_question = "What is Form990 EZ and when should an organiaztion complete Form990 EZ form? And how is it different from Schedule H? Can you show the results in side by side comparison table with headers and also link to the document?"
# user_question = "What does Articles 9 and 25 of the OECD Model Tax Convention state?" # Example question for OECD tool
messages.append({"role": "user", "content": user_question})

# Ensure openai_tools is defined from the previous step (conversion of LlamaIndex tools)
if 'openai_tools' not in globals() or not openai_tools:
    print("OpenAI tool definitions ('openai_tools') not found or are empty. Cannot proceed with tool-using chat loop.")
    # Define dummy tools to prevent crash if previous cell failed, but tool calls won't work
    openai_tools = [{"type": "function", "function": {"name": "dummy_tool", "description": "A dummy tool.", "parameters": {"type": "object", "properties": {"input": {"type": "string"}}, "required": ["input"]}}}]


# --- Main Chat Loop ---
print(f"Starting chat loop for question: {user_question}")

# Keep track of the number of turns to prevent infinite loops
max_turns = 10
turn_count = 0
final_response_content = None # Variable to store the final answer from the model

while turn_count < max_turns:
    turn_count += 1
    print(f"\n--- Turn {turn_count} ---")

    # 2. Send messages to the model using chat_with_tools
    # Ensure chat_with_tools function is defined from a previous cell
    api_response = chat_with_tools(messages, openai_tools)

    # Check if the API call was successful and has choices
    if not api_response or not hasattr(api_response, 'choices') or not api_response.choices:
        print("API call failed or returned no choices.")
        break # Exit loop if API call fails

    # Extract the message from the response
    response_message = api_response.choices[0].message
    print(f"Model response received (Role: {response_message.role})")

    # 3. Check if the response contains tool_calls
    tool_calls = response_message.tool_calls

    if tool_calls:
        print("Model requested tool calls.")
        # Append the model's message (requesting tools) to the messages history
        messages.append(response_message)

        # 4. Iterate through each tool call
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_args_str = tool_call.function.arguments
            tool_call_id = tool_call.id # Get the tool call ID

            print(f"  Tool call requested: {function_name} with args: {function_args_str}")

            # 5. Parse the arguments string
            try:
                function_args = json.loads(function_args_str)
                tool_input = function_args.get("input") # Extract the input argument
                if tool_input is None:
                     print(f"Warning: 'input' argument not found in tool call args for {function_name}.")
                     tool_input = "" # Use empty string if input is missing

            except json.JSONDecodeError:
                print(f"Error decoding tool call arguments JSON for {function_name}: {function_args_str}")
                tool_input = "" # Use empty string or handle as error

            # 6. Identify and execute the corresponding LlamaIndex query engine
            query_engine = query_engine_map.get(function_name)
            tool_output = "" # Initialize tool output

            if query_engine:
                print(f"  Executing tool: {function_name} with input: '{tool_input}'")
                try:
                    # Execute the query using the LlamaIndex engine
                    llama_response = query_engine.query(tool_input)
                    tool_output = str(llama_response) # Convert response to string
                    print(f"  Tool execution successful. Output snippet: '{tool_output[:100]}...'")
                except Exception as e:
                    print(f"  Error executing LlamaIndex tool '{function_name}': {e}")
                    tool_output = f"Error executing tool: {e}" # Provide error message as tool output
            else:
                print(f"  Error: No LlamaIndex query engine found for tool name '{function_name}'.")
                tool_output = f"Error: Tool '{function_name}' not supported or found."

            # 7. Format the output from the LlamaIndex query engine into the required format for the API
            tool_message = {
                "role": "tool",
                "tool_call_id": tool_call_id, # Link the tool output to the specific tool call
                "content": tool_output,
            }

            # 8. Append this tool message to the messages history
            messages.append(tool_message)
            print("  Tool output appended to messages history.")

        # 9. Send the updated messages history back to the chat_with_tools function
        # The loop continues, and the next iteration will send the updated 'messages'
        # The model will then process the tool outputs and generate a response.

    else:
        # 10. If the model responds without tool_calls, it's the final answer
        print("Model responded without tool calls. This is the final answer.")
        final_response_content = response_message.content
        messages.append(response_message) # Append the final response to history
        break # Exit the loop

    # Add a small delay to avoid hitting rate limits too quickly during development
    time.sleep(1)

# After the loop, print the final answer if available
if final_response_content:
    print("\n--- Final Answer ---")
    print(final_response_content)
elif turn_count >= max_turns:
    print("\n--- Chat loop ended due to reaching max turns ---")
    # Optionally print the last message from the model
    if messages:
        print("Last message from model:")
        print(messages[-1])


print("\nChat loop finished.")


## Handle final response

### Subtask:
Process the model's final response after tool execution to extract the answer.


**Reasoning**:
Extract the final answer string from the `content` attribute of the `response_message` object after the loop concludes, and store it in `final_answer_content`. Print this variable.



In [ ]:
# Assume the chat loop from the previous step has just completed.
# The `response_message` variable holds the last message received from the model.
# The `final_response_content` variable was intended to store the final answer.
# The `messages` list holds the entire conversation history.

# Check if the loop terminated because a final answer was received (i.e., no tool calls in the last message)
# and if the last message is not empty.
if response_message and not response_message.tool_calls and response_message.content:
    final_answer_content = response_message.content
    print("\n--- Final Answer (Extracted after loop) ---")
    print(final_answer_content)
elif turn_count >= max_turns:
    print("\n--- Chat loop ended due to reaching max turns ---")
    # If max turns reached, the last message might contain a partial answer or just model thoughts.
    # We can still try to extract content if available, but it might not be a complete final answer.
    if response_message and response_message.content:
         final_answer_content = response_message.content
         print("Last message content:")
         print(final_answer_content)
    else:
         final_answer_content = "Chat loop ended without a final answer."
         print(final_answer_content)
else:
    # Handle other potential loop exit conditions or errors
    final_answer_content = "Chat loop terminated unexpectedly without a final answer."
    print(final_answer_content)

# The final_answer_content variable now holds the extracted final answer or an informative message.
# It can be used for further processing or evaluation.

# Note: The previous cell's code already included printing logic after the loop.
# This cell explicitly focuses on ensuring the extraction and storage in `final_answer_content`.
# The printing is included here to demonstrate the result of the extraction.